In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

df = pd.read_csv('diabetes_skripsi.csv')
df.head()

,puskesmas,kelurahan,umur,jk,merokok,aktivitas_fisik,konsumsi_alkohol,tekanan_darah,bmi,lingkar_perut,pemeriksaan_gula,latitude,longitude,diagnosis
0,Bogor Selatan,Pamoyanan,48.0,0,0,0,0,70.0,18.73,85.0,197,-6.640877,106.793165,1
1,Bogor Selatan,Pamoyanan,75.0,0,0,0,0,70.0,21.64,80.0,155,-6.640877,106.793165,1
2,Bogor Selatan,Pamoyanan,67.0,0,0,1,0,70.0,27.68,84.0,180,-6.640877,106.793165,1
3,Bogor Selatan,Ranggamekar,42.0,0,0,0,0,88.0,31.59,93.5,210,-6.624390,106.791912,1
4,Bogor Selatan,Ranggamekar,45.0,0,0,0,0,80.0,20.94,90.0,240,-6.624390,106.791912,1


In [2]:
# Delete columns 'puskesmas', 'kelurahan', 'longitude', and 'latitude'
df = df.drop(['puskesmas', 'kelurahan', 'longitude', 'latitude'], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   umur              2091 non-null   float64
 1   jk                2091 non-null   int64  
 2   merokok           2091 non-null   int64  
 3   aktivitas_fisik   2091 non-null   int64  
 4   konsumsi_alkohol  2091 non-null   int64  
 5   tekanan_darah     2091 non-null   float64
 6   bmi               2091 non-null   float64
 7   lingkar_perut     2091 non-null   float64
 8   pemeriksaan_gula  2091 non-null   int64  
 9   diagnosis         2091 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 163.5 KB


In [4]:
print(df.columns)

Index(['umur', 'jk', 'merokok', 'aktivitas_fisik', 'konsumsi_alkohol',
       'tekanan_darah', 'bmi', 'lingkar_perut', 'pemeriksaan_gula',
       'diagnosis'],
      dtype='object')


In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
# Define features (X) and target (y)
X = df.drop(['diagnosis'], axis=1)
y = df['diagnosis']

In [7]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [8]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [9]:
# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [11]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
51/51 [==============================] - 16s 59ms/step - loss: 0.6851 - accuracy: 0.5877 - val_loss: 0.6781 - val_accuracy: 0.6127
Epoch 2/50
51/51 [==============================] - 0s 9ms/step - loss: 0.6664 - accuracy: 0.6239 - val_loss: 0.6612 - val_accuracy: 0.6422
Epoch 3/50
51/51 [==============================] - 0s 8ms/step - loss: 0.6456 - accuracy: 0.6276 - val_loss: 0.6384 - val_accuracy: 0.6495
Epoch 4/50
51/51 [==============================] - 0s 8ms/step - loss: 0.6206 - accuracy: 0.6399 - val_loss: 0.6062 - val_accuracy: 0.7255
Epoch 5/50
51/51 [==============================] - 0s 9ms/step - loss: 0.5916 - accuracy: 0.7098 - val_loss: 0.5718 - val_accuracy: 0.7574
Epoch 6/50
51/51 [==============================] - 0s 9ms/step - loss: 0.5556 - accuracy: 0.7350 - val_loss: 0.5329 - val_accuracy: 0.7549
Epoch 7/50
51/51 [==============================] - 0s 9ms/step - loss: 0.5125 - accuracy: 0.7620 - val_loss: 0.4845 - val_accuracy: 0.7892
Epoch 8/50
51/51 [

In [12]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.9764705896377563


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Setelah model dilatih, kita lakukan prediksi pada test set
y_pred_prob = model.predict(X_test)

# Konversi prediksi probabilitas menjadi nilai kelas biner (0 atau 1)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Hitung MAE, RMSE, dan MAPE berdasarkan probabilitas prediksi
mae = mean_absolute_error(y_test, y_pred_prob)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_prob))


# Print hasil evaluasi
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

16/16 [==============================] - 2s 5ms/step
Mean Absolute Error (MAE): 0.05863835963254685
Root Mean Squared Error (RMSE): 0.13048514383333157


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Menghitung metrik klasifikasi
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Menampilkan hasil
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Optional: Tampilkan laporan klasifikasi
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9764705882352941
Precision: 1.0
Recall: 0.9512195121951219
F1 Score: 0.975

Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       264
           1       1.00      0.95      0.97       246

    accuracy                           0.98       510
   macro avg       0.98      0.98      0.98       510
weighted avg       0.98      0.98      0.98       510



In [15]:
# Misalkan data baru dalam bentuk array atau dataframe
new_data = np.array([[39, 1, 0, 0, 0, 92, 23.07, 90, 106]])

# Scale data baru dengan scaler yang sama yang digunakan pada data pelatihan
new_data_scaled = scaler.transform(new_data)

# Reshape data baru untuk menyesuaikan input LSTM (samples, timesteps, features)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Prediksi probabilitas diabetes dengan model yang sudah dilatih
new_prediction_prob = model.predict(new_data_scaled)

# Konversi probabilitas ke prediksi kelas (0 atau 1)
new_prediction_class = (new_prediction_prob > 0.5).astype("int32")

# Tampilkan hasil prediksi
print(f"Probabilitas Diabetes: {new_prediction_prob[0][0]}")
print(f"Prediksi Kelas: {new_prediction_class[0][0]}")  # 1: Diabetes, 0: Non-Diabetes

C:\Users\Soke\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 294ms/step
Probabilitas Diabetes: 0.03010779246687889
Prediksi Kelas: 0


In [16]:
# Setelah melatih, simpan model LSTM dalam format SavedModel
# model.save('model_lstm_saved', save_format='tf')
# model.save('model_lstm_saved.h5')

In [17]:
# import joblib

# # Simpan scaler ke file .pkl
# joblib.dump(scaler, 'scaler.pkl')

In [19]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [20]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)